In [352]:
#Import libraries

mport gzip #for parsing gz files
import pandas as pd
import re 
import inflect #numeric/cardinal

In [353]:
dirPath = r"C:\Study\Projects\NLP\data"
reviewPath = r"reviews_Tools_and_Home_Improvement_5.json.gz"
#reviewPath = r"reviews_Musical_Instruments_5.json.gz"
qaPath = r"C:\Study\Projects\NLP\data" 
path = dirPath + "\\" + reviewPath

In [354]:
dataFile = gzip.open(path, 'r')

In [355]:
# Need this function because large amount of data we are going to load up

# http://jmcauley.ucsd.edu/data/amazon/index.html
# eval function allows text to be parsed as python code
# parse function is a generator with yield
# yield when we want to iterate over a sequence, but don't want to store the entire sequence in memory
def parse(dataFile):
    for each in dataFile:
        yield eval(each)

In [356]:
# for each review for a product in dataFile maps dict[nth] = review (nth)
# enumerate can't be used on a generator
count = 0
productReviews = {}
for eachReview in parse(dataFile):
    productReviews[count] = eachReview
    count = count + 1


In [357]:
len(productReviews)

134476

In [93]:
#productReviews[101718]

In [358]:
productsReviewData = pd.DataFrame.from_dict(productReviews, orient='index')

In [359]:
productsReviewData.shape
productsReviewData.columns
productsReviewData.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A4IL0CLL27Q33,104800001X,D. Brennan,"[0, 1]","I hate it when my shirt collars, not otherwise...",5.0,Perfect for collar stay management,1390953600,"01 29, 2014"
1,A3Q5W5E7TDVLJF,104800001X,funnyc130,"[0, 0]",These little magnets are really powerful for t...,5.0,Neat,1369958400,"05 31, 2013"


In [360]:
reduceData = productsReviewData.loc[:, ['asin', 'reviewerName', 'reviewText', 'helpful', 'overall']]

In [361]:
reduceData = reduceData.loc[reduceData['overall']!=3.0, ['asin', 'reviewerName', 'reviewText', 'helpful', 'overall']]

In [362]:
rating = lambda x: 1 if x > 2.0 else 0

In [363]:
reduceData['rating'] = reduceData['overall'].apply(rating)
unfavorableReviews = list(reduceData.loc[ reduceData['rating'] == 0, 
                                         ['reviewText', 'helpful', 'overall', 'rating']].sort_values(by='overall').index)
#unfavorableReviews
reduceData.loc[ reduceData['rating'] == 0, ['reviewText', 'helpful', 'overall', 'rating']].sort_values(by='overall').head(5)


,reviewText,helpful,overall,rating
134322,Amazing that they designed this thing without ...,"[2, 2]",1.0,0
55236,I was ready to pick up one of these at a woodw...,"[30, 38]",1.0,0
55234,I bought this set from amazon but is not like ...,"[8, 15]",1.0,0
105852,Horrible.Not one blade runs true. Some are as...,"[0, 0]",1.0,0
55216,This product is almost a joke. The case is to...,"[4, 9]",1.0,0


In [364]:
reviewTextProcessing = pd.DataFrame(reduceData.reviewText.copy())

In [365]:
reviewTextProcessing['rating'] = reduceData['rating']
reviewTextProcessing['overall'] = reduceData['overall']

In [366]:
productsReviewData.dtypes
len(productsReviewData)
# reviewText, rating, overall
reviewTextProcessing.shape

(123707, 3)

In [367]:
#reviewTextProcessing.loc[reviewTextProcessing['rating']==0, ['rating', 'overall', 'Processed', 'SentimentModel']]

In [368]:
#https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
import Contractions_Words as contractions
import importlib
importlib.reload(contractions)

contractions_dict = contractions.contractions_dict

#print(contractions_dict.keys())
#print(contractions_dict["s***"])

pattern = '%s' % '|'.join(contractions_dict.keys())
#pattern = r"%s" % pattern
#print(pattern)
contractions_re = re.compile(pattern)
def expand_contractions(s, contractions_dict=contractions_dict):
    #edit for special case like s*** etc
    s = re.sub("\*", "#", s)
    def replace(match):
        #print(type(match), match)
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)


In [369]:
sample1 = "s***"
print(re.sub("\*", "#", sample1))

sample = "I don\'t know what to say nay. I haven\'t. s###"
expand_contractions(sample)
expand_contractions(sample1)

s###


'shit'

In [370]:
from nltk.tokenize import word_tokenize, sent_tokenize
import string

# return both wholeText, paragraph Text
def singleReviewCleanUp(sample):
    #print(sample)
    #sample = sample.decode("utf-8")
    sample = expand_contractions(sample)
    sample = sample.lower()
    
    #for word in sample:
    #    print(lemmatizer.lemmatize(word, wordnet.VERB))
    
    review = sent_tokenize(sample)
    #print(review)
    for count, eachSent in enumerate(review):
        #remove numbers
       eachSent = re.sub("r'\d+/\d+|\d+|\$\d+.\d+", " ", eachSent)
        #remove string
        eachSent = re.sub('[%s]' % re.escape(string.punctuation), ' ', eachSent)
        #word tokens
        #print([lemmatizer.lemmatize(text, wordnet.VERB) for text in eachSent.split()])
        eachSent = word_tokenize(eachSent)
        review[count] = eachSent
        # lemmatize and remove stopWords
        #eachSent = [word for word in eachSent if not word in stopWords]
        #eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent]
        #eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent if not word in stopWords]
        review[count] = " ".join(eachSent)
    wholeText = [" ".join(review) ]
    paragraphText = review
    #return wholeText
    #return paragraphText
    return paragraphText, wholeText

#paragraphText, wholeText = singleReviewCleanUp(sample)
#wholeText, paragraphText
#paragraphText

In [371]:
#Give me back text in a paragraph sentence forms. They do have more values than a clobbed of text based on Polarity Check.
cleanUpTextParagraph = lambda text: singleReviewCleanUp(text)[0]


In [372]:
%%time
reviewTextProcessing['Processed'] = reviewTextProcessing.reviewText.apply(cleanUpTextParagraph)


Wall time: 3min 14s


In [108]:
#sample = " I don't know what to do. I need to test through tough. Do you have any suggestions"
#result = cleanUpTextParagraph(sample)
#result

In [109]:
from textblob import TextBlob

# need to do a data check first
def getPolaritySubjectivity(data):
    #print(data)
    paragraphText = data
    sumPolarity = []
    sumSubjectivity = []
    for each in paragraphText:
        sumPolarity.append(TextBlob(each).sentiment.polarity)
        sumSubjectivity.append(TextBlob(each).sentiment.subjectivity)
    #print("sumPol", sumPolarity)
    return sumPolarity, sumSubjectivity
    #return sum(sumPolarity), sum(sumSubjectivity)

    #print(each, TextBlob(each).sentiment.polarity, TextBlob(each).sentiment.subjectivity)
#(sample2, TextBlob(sample2).sentiment.polarity, TextBlob(sample2).sentiment.subjectivity)


In [110]:
#Return the sume of polarity in a paragraph sentence when give a text in a list form of sentence

getPolarityFromText = lambda text: getPolaritySubjectivity(text)[0]

In [111]:
#for each in sampleText:
#    sample = cleanUpTextParagraph(each)
#    print(sample, getPolarityFromText(sample))

In [112]:
#print(stopWords)

In [113]:
#sentiment = lambda text: TextBlob(text).sentiment.polarity

In [114]:
adjustSentimentModel = lambda x: [sum(x), 0.0] if sum(x) < 0 else [sum(x), 1.0]

In [115]:
%%time
reviewTextProcessing['Sentiment']= reviewTextProcessing.Processed.apply(getPolarityFromText)

Wall time: 7min 26s


In [116]:
%%time
reviewTextProcessing['SentimentModel']= reviewTextProcessing.Sentiment.apply(adjustSentimentModel)

Wall time: 400 ms


In [117]:
#reviewTextProcessing.loc[unfavorableReviews, ['Processed', 'Sentiment', 'SentimentModel']]
#reviewTextProcessing.dtypes

In [373]:
reviewTextProcessing['CleanedJoin'] = reviewTextProcessing['Processed'].apply(', '.join)


In [374]:
reviewTextProcessing.shape
reviewTextProcessing.columns

Index(['reviewText', 'rating', 'overall', 'Processed', 'CleanedJoin'], dtype='object')

In [375]:
#from gensim import corpora
#dictionary = corpora.Dictionary(T)
#print(dictionary.token2id)

In [376]:
reviewTextProcessing['CleanedJoin'].head(5)

0    i hate it when my shirt collars not otherwise ...
1    these little magnets are really powerful for t...
2    i wanted something this small to mount on the ...
3    i use these to magnetize my warhammer k miniat...
4    they are soo freaking annoying, why, you spend...
Name: CleanedJoin, dtype: object

In [378]:
reviewTextProcessing.sort_values(by='rating', inplace=True)

In [379]:
reviewTextProcessing.head(5)

,reviewText,rating,overall,Processed,CleanedJoin
99410,Bought one and loved it! Ordered another two.O...,0,1.0,"[bought one and loved it, ordered another two ...","bought one and loved it, ordered another two o..."
110570,Not made in the usa.. I wouldn't dare use this...,0,1.0,[not made in the usa i would not dare use this...,not made in the usa i would not dare use this ...
55485,"These are now made in China, not England. The...",0,2.0,"[these are now made in china not england, the ...","these are now made in china not england, the h..."
90164,Mine arrived with a bend in the bracket that s...,0,1.0,[mine arrived with a bend in the bracket that ...,mine arrived with a bend in the bracket that s...
55476,The picture shows the Marples that used to be ...,0,1.0,[the picture shows the marples that used to be...,the picture shows the marples that used to be ...


In [380]:
size = reviewTextProcessing.loc[reviewTextProcessing['rating'] == 0, :].shape[0]
size

10105

In [383]:
reviewTextProcessing = reviewTextProcessing.iloc[0:size*5, :]


In [382]:
additionalNegatives = reviewTextProcessing.iloc[0:size, :].copy()
additionalNegatives.head(5)

,reviewText,rating,overall,Processed,CleanedJoin
99410,Bought one and loved it! Ordered another two.O...,0,1.0,"[bought one and loved it, ordered another two ...","bought one and loved it, ordered another two o..."
110570,Not made in the usa.. I wouldn't dare use this...,0,1.0,[not made in the usa i would not dare use this...,not made in the usa i would not dare use this ...
55485,"These are now made in China, not England. The...",0,2.0,"[these are now made in china not england, the ...","these are now made in china not england, the h..."
90164,Mine arrived with a bend in the bracket that s...,0,1.0,[mine arrived with a bend in the bracket that ...,mine arrived with a bend in the bracket that s...
55476,The picture shows the Marples that used to be ...,0,1.0,[the picture shows the marples that used to be...,the picture shows the marples that used to be ...


In [384]:
testSample = pd.concat([reviewTextProcessing, additionalNegatives])

In [385]:

import itertools

largeText = testSample.CleanedJoin.to_list()
#largeText = reviewTextProcessing.reviewText.to_list() # Not too good accuracy around 116 missed with %86.94

len(largeText)

60630

In [273]:
#sentiment = reviewTextProcessing['SentimentModel'].to_list()

KeyError: 'SentimentModel'

In [170]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

num_words = 15000


In [386]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(largeText)
#tokenizer.word_index

In [298]:
from sklearn.model_selection import train_test_split

In [387]:
X = largeText
y = testSample['rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=310)

x_train_text = X_train
x_test_text = X_test

x_train_tokens = tokenizer.texts_to_sequences(x_train_text)
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)

In [388]:
len(y_train), len(y_test), len(x_train_text), len(x_test_text)

(45472, 15158, 45472, 15158)

In [128]:
#x_train_tokens[1], x_train_text[1]

In [389]:
import numpy as np

x_train_text[1]
np.array(x_train_tokens[1])
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)


In [390]:
np.mean(num_tokens), np.max(num_tokens)


(109.2761339270988, 3563)

In [391]:
np.std(num_tokens)

131.30618335778905

In [392]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens) 
max_tokens


371

In [393]:
np.sum(num_tokens < max_tokens) / len(num_tokens)


0.9599373247567211

In [394]:
pad = 'pre'
#pad = 'post'
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_train_pad.shape, x_test_pad.shape


((45472, 371), (15158, 371))

In [395]:
for i in range(0, 1):
    print(x_train_pad[i])


[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [178]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [179]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [180]:
x_train_text[1]


'these little magnets are really powerful for there size, i am using them to make secret compartments in custom made boxes, each one hols about of a pound'

In [181]:
tokens_to_string(x_train_tokens[1])


'these little magnets are really powerful for there size i am using them to make secret compartments in custom made boxes each one about of a pound'

In [55]:
from keras import Sequential
from keras.layers import Dense, Embedding, Dropout
from keras.models import Sequential
from keras.layers import Bidirectional
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import GlobalMaxPooling1D
from keras.optimizers import Adam

Using TensorFlow backend.


In [183]:
import tensorflow as tf
tf.__version__


'1.13.2'

In [56]:
optimizer = Adam(lr=1e-3)


Instructions for updating:
Colocations handled automatically by placer.


In [396]:
model_cnn = Sequential()
parameters = Embedding(num_words, 200, input_length=max_tokens)
model_cnn.add(parameters)
model_cnn.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(256, activation='relu'))
model_cnn.add(Dense(1, activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model_cnn.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 371, 200)          3000000   
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 370, 100)          40100     
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 100)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 256)               25856     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 257       
Total params: 3,066,213
Trainable params: 3,066,213
Non-trainable params: 0
_________________________________________________________________


In [397]:
index = len(x_train_pad)
index
len(y_train)

45472

In [410]:
%%time
model_cnn.fit(x_train_pad, y_train, validation_split=0.05, epochs=5, batch_size=74)
#model.fit(x_train_pad, y_train,validation_split=0.05, epochs=3, batch_size=74)

Train on 43198 samples, validate on 2274 samples
Epoch 1/5
43198/43198 [==============================] - 205s 5ms/step - loss: 0.0085 - acc: 0.9982 - val_loss: 0.2812 - val_acc: 0.9371
Epoch 2/5
43198/43198 [==============================] - 195s 5ms/step - loss: 0.0038 - acc: 0.9994 - val_loss: 0.3095 - val_acc: 0.9362
Epoch 3/5
43198/43198 [==============================] - 204s 5ms/step - loss: 0.0030 - acc: 0.9995 - val_loss: 0.3362 - val_acc: 0.9340
Epoch 4/5
43198/43198 [==============================] - 209s 5ms/step - loss: 0.0034 - acc: 0.9993 - val_loss: 0.3726 - val_acc: 0.9252
Epoch 5/5
43198/43198 [==============================] - 203s 5ms/step - loss: 0.0034 - acc: 0.9991 - val_loss: 0.4185 - val_acc: 0.9292
Wall time: 16min 55s


In [409]:
#more negative training
#model_cnn.fit(x_negative_pad, y_negative, validation_split=0.05, epochs=3, batch_size=36)

In [411]:
%%time
result = model_cnn.evaluate(x_test_pad, y_test)

15158/15158 [==============================] - 25s 2ms/step
Wall time: 24.8 s


In [412]:
print("Accuracy: {0:.2%}".format(result[1]))


Accuracy: 94.14%


In [413]:
%%time
#y_pred = model_cnn_02.predict(x=x_test_pad[0:1000])
y_pred = model_cnn.predict(x=x_test_pad)
y_pred = y_pred.T[0]



Wall time: 22.1 s


In [414]:
pd.DataFrame(y_pred).describe()

,0
count,15158.000000
mean,0.668828
std,0.459213
min,0.000000
25%,0.001779
50%,0.999994
75%,1.000000
max,1.000000


In [ ]:
#y_pred = y_pred.T[0]
#list(np.where(y_pred.T[0]))

In [ ]:
#y_pred

In [415]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])


In [416]:
len(cls_pred),

(15158,)

In [417]:
accuracy = (cls_pred==y_test).mean()
accuracy

0.9413511017284603

In [418]:
sample = len(cls_pred)
sample

15158

In [419]:
tn, fp, fn, tp = confusion_matrix(y_test, cls_pred).ravel()
tn, fp, fn, tp

(4597, 461, 428, 9672)

In [408]:
len(np.where(y_test==0)[0]),len(np.where(y_test==1)[0])

(5058, 10100)

In [ ]:
len(np.where(y_train==0)[0]),len(np.where(y_train==1)[0])

In [85]:
cls_true = np.array(y_test)
#cls_true = np.where(y_test[0:1000])
#cls_true

In [86]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]
len(incorrect)

129

In [194]:
correct = np.where(cls_pred == cls_true)
#correct[0]

In [195]:
len(incorrect)


48

In [140]:
#np.where(y_pred.T)[0]

In [69]:
result_location = np.where(y_pred.T)[0]  + index
CNNresult = reviewTextProcessing.loc[list(result_location),  ['reviewText', 'overall', 'rating', 'Sentiment' , 'SentimentModel']]
#CNNresult.head(2)
#CNNresult.shape
#result_location
#result_location
CNNresult.head(5)

,reviewText,overall,rating,Sentiment,SentimentModel
7695,This is my first tube amp and after doing a TO...,5.0,1,"[0.36481481481481476, 0.275, -0.16666666666666...","[1.4361772486772488, 1.0]"
7696,"This is an amazing amp. It works great, a lot ...",5.0,1,"[0.6000000000000001, 0.10312500000000001, 0.58...","[2.092013888888889, 1.0]"
7697,I've had this for about 3 years and I agree wi...,4.0,1,"[0.55, 0.6, 0.25, 0.0, 0.5, -0.05, 0.0, -0.125...","[5.091515151515152, 1.0]"
7698,At the $600 price point for an all tube amp PL...,5.0,1,"[-1.0, 0.011388888888888888, -0.075, 0.0, 0.0]","[-1.063611111111111, 0.0]"
7699,"I play in our band at church and as such, I've...",5.0,1,"[0.0, 0.25555555555555554, 0.0, -0.15, 0.2, 0....","[5.556873015873016, 1.0]"


In [70]:
CNNresult['CNN_Sentiment'] = cls_pred


In [71]:
sumSentiment = lambda x: x[1]

In [72]:
CNNresult.columns = ['TextBlobAnalysis' if x=='Sentiment' else x for x in CNNresult.columns]
CNNresult.columns = ['TextBlob_Sentiment' if x=='SentimentModel' else x for x in CNNresult.columns]
CNNresult['TextBlob_Sentiment'] = CNNresult['TextBlob_Sentiment'].apply(sumSentiment)

In [73]:
CNNresult.CNN_Sentiment = CNNresult.CNN_Sentiment.astype(int)
CNNresult.TextBlob_Sentiment = CNNresult.TextBlob_Sentiment.astype(int)

In [74]:
CNNresult.loc[CNNresult.overall >= 3.0 , :].to_csv("Rating3.0above_CNN.csv")
CNNresult.loc[CNNresult.overall < 3.0 , :].to_csv("RatingBelow3.0_CNN.csv")

In [75]:
from sklearn.metrics import confusion_matrix

y_true = CNNresult.rating.to_list()
cnn_pred = CNNresult.CNN_Sentiment.to_list()
textBlob_pred = CNNresult.TextBlob_Sentiment.to_list()


In [205]:
#textBlob_pred

In [76]:
tn, fp, fn, tp = confusion_matrix(y_true, cnn_pred).ravel()
tn, fp, fn, tp

(3, 126, 5, 2432)

In [77]:
tn, fp, fn, tp = confusion_matrix(y_true, textBlob_pred).ravel()
tn, fp, fn, tp

(34, 95, 139, 2298)

In [ ]:
idx = incorrect[1]
print(type(idx))
for each in incorrect[:10]:
    idx = each
    text = x_test_text[idx]
    print(idx, text, cls_pred[idx], cls_true[idx], " BlobText ",
         TextBlob(text).sentiment.polarity)


In [ ]:
pd.options.display.max_colwidth = 500

#productsReviewData.cleanedText.head(2)